In [1]:
import httpx
from IPython.display import display, HTML, display_html
from IPython.display import Image
from PIL import Image as PILImage
from io import BytesIO, StringIO
import bs4
import pandas as pd
import pytesseract

import sys
sys.path.append('..')
from app.config import settings


In [2]:
SRM_STUDENT_PORTAL_URI = "https://sp.srmist.edu.in/srmiststudentportal/students/loginManager/youLogin.jsp"
SRM_STUDENT_PORTAL_GET_CAPTCHA_URI = "https://sp.srmist.edu.in/srmiststudentportal/captchas"
ATTENDANCE_PAGE_URI = "https://sp.srmist.edu.in/srmiststudentportal/students/report/studentAttendanceDetails.jsp"
MONTHLY_ATTENDANCE_PAGE_URI = "https://sp.srmist.edu.in/srmiststudentportal/students/report/studentAttendanceDetailsInner.jsp"
TIMETABLE_PAGE_URI = "https://sp.srmist.edu.in/srmiststudentportal/students/report/studentTimeTableDetails.jsp"


In [3]:
client = httpx.AsyncClient()

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:122.0) Gecko/20100101 Firefox/122.0',
}


## Login Page

In [4]:
# Make GET request to SRM Student Portal
await client.get(SRM_STUDENT_PORTAL_URI, headers=headers)



2024-05-08 11:15:44,320 - httpx - INFO - HTTP Request: GET https://sp.srmist.edu.in/srmiststudentportal/students/loginManager/youLogin.jsp "HTTP/1.1 200 200"


<Response [200 200]>

In [5]:
# Get Captcha
captcha_response = await client.get(SRM_STUDENT_PORTAL_GET_CAPTCHA_URI, headers=headers)
captcha_text = pytesseract.image_to_string(PILImage.open(BytesIO(captcha_response.content))).strip()

print(f'Capcha: {captcha_text}')
display(Image(data=captcha_response.content))


2024-05-08 11:15:44,376 - httpx - INFO - HTTP Request: GET https://sp.srmist.edu.in/srmiststudentportal/captchas "HTTP/1.1 200 200"


Capcha: gmxrNw


In [6]:
# Login
response = await client.post(
    SRM_STUDENT_PORTAL_URI,
    data={
        "txtPageAction": "1",
        "txtAN": settings.SRM_PORTAL_USERNAME,
        "txtSK": settings.SRM_PORTAL_PASSWORD,
        "hdnCaptcha": captcha_text,
    }
)

# Check for Login Error
if "Login Error : Invalid net id or password" in response.text:
    raise Exception("Invalid Username or Password")

# Check for Captcha Error, If so then try again
if "Invalid Captcha...." in response.text:
    raise Exception("Invalid Captcha")
    
# Check status code
# status should be `302` because after successful login it redirects to another page 
# if response.status_code != 302:
#     raise Exception("SRM Student Portal is Down")


2024-05-08 11:15:44,569 - httpx - INFO - HTTP Request: POST https://sp.srmist.edu.in/srmiststudentportal/students/loginManager/youLogin.jsp "HTTP/1.1 302 302"


In [8]:
display(HTML(response.text))


## Attendance

In [9]:
response = await client.post(ATTENDANCE_PAGE_URI, headers=headers)


2024-05-08 11:16:21,427 - httpx - INFO - HTTP Request: POST https://sp.srmist.edu.in/srmiststudentportal/students/report/studentAttendanceDetails.jsp "HTTP/1.1 200 200"


In [128]:
soup = bs4.BeautifulSoup(response.text, 'html.parser')
table = soup.findAll('table', class_='table')[0]

df = pd.read_html(StringIO(str(table)))[0]

# Remove rows whoose `Code` matches below list
code_blacklist = ['CL', 'Total']
df = df[~df['Code'].isin(code_blacklist)]
df


,Code,Description,Max. hours,Att. hours,Absent hours,Average %,OD/ML Percentage,Total Percentage
0,21CSC204J,DESIGN AND ANALYSIS OF ALGORITHMS,83,72,11,86.75,0.0,86.75
1,21CSC205P,DATABASE MANAGEMENT SYSTEMS,82,66,16,80.49,0.0,80.49
2,21CSC206T,ARTIFICIAL INTELLIGENCE,69,61,8,88.41,0.0,88.41
3,21CSE251T,DIGITAL IMAGE PROCESSING,55,45,10,81.82,0.0,81.82
4,21LEM202T,UNIVERSAL HUMAN VALUES - II: UNDERSTANDING HAR...,29,22,7,75.86,0.0,75.86
5,21MAB204T,PROBABILITY AND QUEUEING THEORY,74,61,13,82.43,0.0,82.43
6,21PDH201T,SOCIAL ENGINEERING,28,24,4,85.71,0.0,85.71
7,21PDM202L,CRITICAL AND CREATIVE THINKING SKILLS,27,25,2,92.59,0.0,92.59


In [ ]:
soup = bs4.BeautifulSoup(response.text, 'html.parser')
table = soup.findAll('table', class_='table')[1]

df = pd.read_html(StringIO(str(table)))[0]
# month_map = {'JAN': 1, 'FEB': 2,'MAR': 3, 'APR': 4, 'MAY': 5, 'JUN': 6, 'JUL': 7, 'AUG': 8, 'SEP': 9, 'OCT': 10, 'NOV': 11, 'DEC': 12}
# Extracting the month and year where there are entries
months_years = []
for index, row in df.iterrows():
    month, year = row['Month / Year'].split(' / ')
    if row[['Present', 'Absent']].sum() > 0:
        months_years.append([int(year), pd.to_datetime(month, format='%b').month])

print(months_years)


In [ ]:
df = pd.read_html(StringIO(str(table)))[0]
for index, row in df.iterrows():
    mon, year = row['Month / Year'].split(' / ')
    print(mon, year)


In [ ]:
MONTHLY_ATTENDENCE_PAGE_URI = 'https://sp.srmist.edu.in/srmiststudentportal/students/report/studentAttendanceDetailsInner.jsp'
response = session.post(
    MONTHLY_ATTENDENCE_PAGE_URI,
    data={'ids': 1, 'attendanceMonth': 1, 'attendanceYear': 2024},
)
lst = []
import ast
df = pd.read_html(StringIO(str(response.text)))[0]
lst.extend(ast.literal_eval(df.to_json(index=False, orient='records')))
lst


## Timetable

In [10]:
response = await client.post(
    'https://sp.srmist.edu.in/srmiststudentportal/students/report/studentTimeTableDetails.jsp',
)


2024-05-08 11:16:47,109 - httpx - INFO - HTTP Request: POST https://sp.srmist.edu.in/srmiststudentportal/students/report/studentTimeTableDetails.jsp "HTTP/1.1 200 200"


In [19]:
soup = bs4.BeautifulSoup(response.text, 'html.parser')
timetable, legends = soup.findAll('table', class_='table')
timetable = pd.read_html(StringIO(str(timetable)))[0]
legends = pd.read_html(StringIO(str(legends)))[0]
timetable


,Day order,1,2,3,4,5,6,7,8
,Day order,08:30-09:20,09:20-10:10,10:20-11:10,11:10-12:00,01:30-02:20,02:20-03:10,03:10-04:00,04:00-04:50
0,Monday,21CSC204J,21CSC205P,21CSC206T,21MAB204T,21CSE251T 21CSE271T,21PDM202L,21PDH201T,-
1,Tuesday,21CSC204J,21CSC204J,-,21CSC206T,21CSE251T 21CSE271T,21MAB204T,21CSC205P,-
2,Wednesday,21CSC205P,21CSC204J,21MAB204T,21LEM202T,21CSE251T 21CSE271T,21CSC206T,21CSC204J,-
3,Thursday,21CSC205P,21CSC205P,-,21MAB204T,21CSE251T 21CSE271T,21MAB204T,21CSC206T,-
4,Friday,21LEM202T,21PDM202L,21CSC206T,CL,21CSC205P,21CSC204J,21PDH201T,-
5,Saturday,-,-,-,-,-,-,-,-


In [20]:
# Clean the timetable dataframe
timetable.replace('-', pd.NA, inplace=True)
timetable.columns = timetable.columns.droplevel(0)
timetable.rename(columns={'Day order': 'Day'}, inplace=True)
timetable


,Day,08:30-09:20,09:20-10:10,10:20-11:10,11:10-12:00,01:30-02:20,02:20-03:10,03:10-04:00,04:00-04:50
0,Monday,21CSC204J,21CSC205P,21CSC206T,21MAB204T,21CSE251T 21CSE271T,21PDM202L,21PDH201T,<NA>
1,Tuesday,21CSC204J,21CSC204J,<NA>,21CSC206T,21CSE251T 21CSE271T,21MAB204T,21CSC205P,<NA>
2,Wednesday,21CSC205P,21CSC204J,21MAB204T,21LEM202T,21CSE251T 21CSE271T,21CSC206T,21CSC204J,<NA>
3,Thursday,21CSC205P,21CSC205P,<NA>,21MAB204T,21CSE251T 21CSE271T,21MAB204T,21CSC206T,<NA>
4,Friday,21LEM202T,21PDM202L,21CSC206T,CL,21CSC205P,21CSC204J,21PDH201T,<NA>
5,Saturday,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


In [21]:
# Clean the legends dataframe
legends.rename(columns={'Code': 'SubCode', 'Description': 'SubName'}, inplace=True)
legends


,SubCode,SubName
0,21CSC204J,DESIGN AND ANALYSIS OF ALGORITHMS
1,21CSC205P,DATABASE MANAGEMENT SYSTEMS
2,21CSC206T,ARTIFICIAL INTELLIGENCE
3,21CSE251T,DIGITAL IMAGE PROCESSING
4,21CSE271T,PROGRAMMING IN JAVA
5,21LEM202T,UNIVERSAL HUMAN VALUES - II: UNDERSTANDING HAR...
6,21MAB204T,PROBABILITY AND QUEUEING THEORY
7,21PDH201T,SOCIAL ENGINEERING
8,21PDM202L,CRITICAL AND CREATIVE THINKING SKILLS
9,CL,CLASS IN CHARGE


In [46]:
from app.utils import database, models
from sqlalchemy.orm import Session

db: Session = next(database.get_db())
user = db.query(models.Subjects).filter(models.Subjects.SubjectCode == '21CS206T').first()
user


In [47]:
# Create a map of subject name to subject code
sub_map = {}
for data in legends.to_dict(orient='records'):
    res = db.query(models.Subjects).filter(models.Subjects.SubjectCode == data['SubCode'].strip()).first()
    if res:
        sub_map[res.SubjectCode] = res.Alias
    else:
        sub_map[data['SubCode'].strip()] = data['SubName'].strip()
sub_map


{'21CSC204J': 'DESIGN AND ANALYSIS OF ALGORITHMS',
 '21CSC205P': 'DATABASE MANAGEMENT SYSTEMS',
 '21CSC206T': 'AI',
 '21CSE251T': 'DIGITAL IMAGE PROCESSING',
 '21CSE271T': 'PROGRAMMING IN JAVA',
 '21LEM202T': 'UHV',
 '21MAB204T': 'PROBABILITY AND QUEUEING THEORY',
 '21PDH201T': 'SOCIAL ENGINEERING',
 '21PDM202L': 'CRITICAL AND CREATIVE THINKING SKILLS',
 'CL': 'CLASS IN CHARGE'}

In [23]:
# Replace the subject name with subject code in the timetable dataframe
for idx, row in timetable.iterrows():
    for col in timetable.columns:
        # If there is a single subject code in the cell
        if row[col] in sub_map:
            timetable.at[idx, col] = sub_map[row[col]]
        
        # For electives where there are multiple subject codes
        elif isinstance(row[col], str) and len(row[col].split(' ')) > 1:
            sub_codes = row[col].split(' ')
            sub_names = [sub_map[sub_code] for sub_code in sub_codes]
            timetable.at[idx, col] = ' / '.join(sub_names)
timetable


,Day,08:30-09:20,09:20-10:10,10:20-11:10,11:10-12:00,01:30-02:20,02:20-03:10,03:10-04:00,04:00-04:50
0,Monday,DESIGN AND ANALYSIS OF ALGORITHMS,DATABASE MANAGEMENT SYSTEMS,ARTIFICIAL INTELLIGENCE,PROBABILITY AND QUEUEING THEORY,DIGITAL IMAGE PROCESSING / PROGRAMMING IN JAVA,CRITICAL AND CREATIVE THINKING SKILLS,SOCIAL ENGINEERING,<NA>
1,Tuesday,DESIGN AND ANALYSIS OF ALGORITHMS,DESIGN AND ANALYSIS OF ALGORITHMS,<NA>,ARTIFICIAL INTELLIGENCE,DIGITAL IMAGE PROCESSING / PROGRAMMING IN JAVA,PROBABILITY AND QUEUEING THEORY,DATABASE MANAGEMENT SYSTEMS,<NA>
2,Wednesday,DATABASE MANAGEMENT SYSTEMS,DESIGN AND ANALYSIS OF ALGORITHMS,PROBABILITY AND QUEUEING THEORY,UNIVERSAL HUMAN VALUES - II: UNDERSTANDING HAR...,DIGITAL IMAGE PROCESSING / PROGRAMMING IN JAVA,ARTIFICIAL INTELLIGENCE,DESIGN AND ANALYSIS OF ALGORITHMS,<NA>
3,Thursday,DATABASE MANAGEMENT SYSTEMS,DATABASE MANAGEMENT SYSTEMS,<NA>,PROBABILITY AND QUEUEING THEORY,DIGITAL IMAGE PROCESSING / PROGRAMMING IN JAVA,PROBABILITY AND QUEUEING THEORY,ARTIFICIAL INTELLIGENCE,<NA>
4,Friday,UNIVERSAL HUMAN VALUES - II: UNDERSTANDING HAR...,CRITICAL AND CREATIVE THINKING SKILLS,ARTIFICIAL INTELLIGENCE,CLASS IN CHARGE,DATABASE MANAGEMENT SYSTEMS,DESIGN AND ANALYSIS OF ALGORITHMS,SOCIAL ENGINEERING,<NA>
5,Saturday,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


In [24]:
class_time = ['Day', '9:20-10:10', '10:10-11:00', '11:10-12:00', '12:00-12:50', '2:00-2:50', '2:50-3:40', '3:40-4:30']

timetable.drop('04:00-04:50', axis=1, inplace=True)
timetable.columns = class_time
timetable.drop(5, inplace=True)
timetable


,Day,9:20-10:10,10:10-11:00,11:10-12:00,12:00-12:50,2:00-2:50,2:50-3:40,3:40-4:30
0,Monday,DESIGN AND ANALYSIS OF ALGORITHMS,DATABASE MANAGEMENT SYSTEMS,ARTIFICIAL INTELLIGENCE,PROBABILITY AND QUEUEING THEORY,DIGITAL IMAGE PROCESSING / PROGRAMMING IN JAVA,CRITICAL AND CREATIVE THINKING SKILLS,SOCIAL ENGINEERING
1,Tuesday,DESIGN AND ANALYSIS OF ALGORITHMS,DESIGN AND ANALYSIS OF ALGORITHMS,<NA>,ARTIFICIAL INTELLIGENCE,DIGITAL IMAGE PROCESSING / PROGRAMMING IN JAVA,PROBABILITY AND QUEUEING THEORY,DATABASE MANAGEMENT SYSTEMS
2,Wednesday,DATABASE MANAGEMENT SYSTEMS,DESIGN AND ANALYSIS OF ALGORITHMS,PROBABILITY AND QUEUEING THEORY,UNIVERSAL HUMAN VALUES - II: UNDERSTANDING HAR...,DIGITAL IMAGE PROCESSING / PROGRAMMING IN JAVA,ARTIFICIAL INTELLIGENCE,DESIGN AND ANALYSIS OF ALGORITHMS
3,Thursday,DATABASE MANAGEMENT SYSTEMS,DATABASE MANAGEMENT SYSTEMS,<NA>,PROBABILITY AND QUEUEING THEORY,DIGITAL IMAGE PROCESSING / PROGRAMMING IN JAVA,PROBABILITY AND QUEUEING THEORY,ARTIFICIAL INTELLIGENCE
4,Friday,UNIVERSAL HUMAN VALUES - II: UNDERSTANDING HAR...,CRITICAL AND CREATIVE THINKING SKILLS,ARTIFICIAL INTELLIGENCE,CLASS IN CHARGE,DATABASE MANAGEMENT SYSTEMS,DESIGN AND ANALYSIS OF ALGORITHMS,SOCIAL ENGINEERING


In [25]:
new_timetable = {}
for item in timetable.to_dict(orient='records'):
    day = ''
    for key, value in item.items():
        if key == 'Day':
            day = value
            new_timetable[day] = {}
        else:
            new_timetable[day][key] = value
new_timetable


{'Monday': {'9:20-10:10': 'DESIGN AND ANALYSIS OF ALGORITHMS',
  '10:10-11:00': 'DATABASE MANAGEMENT SYSTEMS',
  '11:10-12:00': 'ARTIFICIAL INTELLIGENCE',
  '12:00-12:50': 'PROBABILITY AND QUEUEING THEORY',
  '2:00-2:50': 'DIGITAL IMAGE PROCESSING / PROGRAMMING IN JAVA',
  '2:50-3:40': 'CRITICAL AND CREATIVE THINKING SKILLS',
  '3:40-4:30': 'SOCIAL ENGINEERING'},
 'Tuesday': {'9:20-10:10': 'DESIGN AND ANALYSIS OF ALGORITHMS',
  '10:10-11:00': 'DESIGN AND ANALYSIS OF ALGORITHMS',
  '11:10-12:00': None,
  '12:00-12:50': 'ARTIFICIAL INTELLIGENCE',
  '2:00-2:50': 'DIGITAL IMAGE PROCESSING / PROGRAMMING IN JAVA',
  '2:50-3:40': 'PROBABILITY AND QUEUEING THEORY',
  '3:40-4:30': 'DATABASE MANAGEMENT SYSTEMS'},
 'Wednesday': {'9:20-10:10': 'DATABASE MANAGEMENT SYSTEMS',
  '10:10-11:00': 'DESIGN AND ANALYSIS OF ALGORITHMS',
  '11:10-12:00': 'PROBABILITY AND QUEUEING THEORY',
  '12:00-12:50': 'UNIVERSAL HUMAN VALUES - II: UNDERSTANDING HARMONY AND ETHICAL HUMAN CONDUCT',
  '2:00-2:50': 'DIGITAL 

## Benchmarks

In [ ]:
# import requests
# url = "http://127.0.0.1:8000/get_attendance_details"
# payload = {'username': 'dm0359', 'password': 'Mymail@06'}


In [ ]:
# %%timeit -n 1 -r 20
# response = requests.request("POST", url, data=payload)
